In [3]:
html = 'SmartTable All instances of Pathways in Escherichia coli K-12 substr. MG1655.html'

In [87]:
headers = []
START_FLAG = 'title="Click to edit">'
def parseHeader(line: str):
    s = line.index(START_FLAG)
    e = line.index('</span><span')
    # print(line[s+len(START_FLAG):e])
    headers.append(line[s+len(START_FLAG):e])

mmap = []
def parseRow(line: str):
    def local(q, i=0):
        s = line
        if q in s:
            st = s.index(q, i)
            print(st, s[st-25:st+100])

        return '/'

    i = 0
    def cut(start, end):
        nonlocal i
        i = line.index(start, i) + len(start)
        return line[i:line.index(end, i)]

    pathway = cut(';b&gt;Pathway&lt;/b&gt;: ', '&lt;br&gt;')

    genes = []
    while True:
        try:
            genes.append(cut('&lt;b&gt;Gene:&lt;/b&gt;', '&lt;br&gt;'))
        except ValueError:
            break
    genes = [(g[0], g[1:]) for g in [g.strip().split('   ') for g in genes]]

    id = cut('<td valign="top" align="none">', '<br></td>')
    common_name = cut('<td valign="top" align="none">', '<br></td>')

    reactions = []
    while True:
        try:
            ec = cut('Reaction:&lt;/b&gt; EC ', ' &lt;b&gt;')
            description = cut('ID:&lt;/b&gt; ', '&lt;br&gt; &lt;b&gt;')
            enzymes = cut('&lt;b&gt;Enzyme:&lt;/b&gt; ', ', WIDTH, 500);')
            reactions.append((ec, description, enzymes))
        except ValueError:
            break

    mmap.append({
        'pathway': pathway,
        'genes': genes,
        'id': id,
        'common_name': common_name,
        'reactions': reactions
    })

parsers = [
    (lambda line: not (line.startswith('<span') and START_FLAG in line), lambda l: l), # do nothing
    (lambda line: line.startswith('<span') and START_FLAG in line, parseHeader),
    (lambda line: line.startswith('<tr'), parseRow),
    (lambda line: line, None)
]

with open(html, 'r') as raw:
    state = 0
    checker, parser = None, None
    for line in raw:
        if checker is None or not checker(line):
            checker, parser = parsers[state]
            state += 1
        if parser is None: break

        parser(line)

len(mmap)

442

In [88]:
import numpy as np

np.save('metacyc_K-12_MG1655', mmap)